## Demande Loic 01/06 : fournir tableau de données pour visualisation principale n3 pour aider à l'écriture

#### Tableau 1 (données Navigo) : 
pour chaque port DFLR en 1789 :
- nombre de bateaux sortis du port 
- tonnage moyen de ces bateaux

#### Tableau 2.1 : (données Navigo, je ne sais pas encore comment ça se calcule)
pour chaque port DFLR en 1787 (pourrait se calculer correctement en 89 mais problème de cohérence des données avec tableau 2.2) : 
- indice de centralité dans le réseau des flux de la région

#### Tableau 2.2 : (données Navigo, je ne sais pas encore comment ça se calcule)
pour ports de La Rochelle, Bordeaux, Le Havre, Nantes en 1787 : 
- indice de centralité dans le réseau des flux de la France

#### Tableau 3.1.1 (données Navigo) : 
pour chaque port DFLR en 1789 (vérifier si ça change pas trop avec 87 mais normalement pas de soucis pour 89) : 
- tonnage cumulé à destination de l’extérieur de la région
- tonnage cumulé à destination de l’hinterland de la région

#### Tableau 3.1.2 (données Navigo) : 
pour les bureaux (agrégation de ports : retrouver les alignements ? pour l'instant je peux raisonner au niveau des amirautés) de La Rochelle, Bordeaux, Le Havre, Nantes en 1789 : 
- tonnage cumulé à destination de l’extérieur de la région
- tonnage cumulé à destination de l’hinterland (définir l'hinterland de chaque région) de la région

#### Tableau 3.2.1 (données Toflit) : 
pour bureau DFLR en 1789 : 
- valeur cumulée exports produits d'origine extérieure à la région
- valeur cumulée exports produits originaires de l’hinterland (PASA)

#### Tableau 3.2.2 (données Toflit) : 
pour bureaux de La Rochelle, Bordeaux, Le Havre, Nantes en 1789 : 
- valeur cumulée exports produits d'origine extérieure à la région
- valeur cumulée exports produits originaires de l’hinterland

In [ ]:
# imports
import json 
import csv
from poitousprint import Portic, Toflit
from vega import VegaLite
import pandas as pd

portic_client = Portic()
toflit_client = Toflit()

#### Tableau 1 (données Navigo) : 
pour chaque port DFLR en 1789 :
- nombre de bateaux passés par le port 
- tonnage moyen de ces bateaux

In [ ]:
# collecte des pointcalls sortis des ports de la DFLR en 1789 
pointcalls_ports_dflr_1789 = portic_client.get_pointcalls(
    year = 1789, 
    ferme_direction = 'La Rochelle', 
    pointcall_action = 'Out' 
)

print("nombre de pointcalls sortis des ports de la DFLR en 1789  :", len(pointcalls_ports_dflr_1789))

In [ ]:
ports_set = set() 
for p in pointcalls_ports_dflr_1789:
    ports_set.add(p['pointcall'])
# print("nb de ports différents trouvés dans la DFLR : ", len(ports_set), " : ", ports_set)

In [ ]:
ports = {}

for p in ports_set:
    ports[p] = {
        "port": p,
        "nb_pointcalls_out":0,
        "cumulated_tonnage": 0
    }


for p in pointcalls_ports_dflr_1789: 
    port = p['pointcall']
    if p['tonnage'] is not None:
        tonnage = int(p['tonnage'])
    else:
        tonnage = 0
    ports[port]['cumulated_tonnage'] += tonnage  
    ports[port]['nb_pointcalls_out'] += 1  
    
part_by_port = []
for port, values in ports.items():
    part_by_port.append({
        "port": port,
        "type": "nombre de navires sortis du port en 1789",
        "portion_nb_boats": values['nb_pointcalls_out'] 
    })
    part_by_port.append({
        "port": port,
        "type": "tonnage cumulé des navires sortis du port en 1789",
        "portion_tonnage": values['cumulated_tonnage']
    })
  
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Distribution du nombre de navires sortis des ports de la DFLR en 1789",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "port",
            "type": "ordinal", 
            "title": "port"
        },
        "y": {
            "type": "quantitative",
            "field": "portion_nb_boats",
            "title": "flux sortants"
        },
        "color": {
            "type": "nominal",
            "field": "type",
            "title": "Type de quantification",
            "scale": {
                 "range": ["green"]
            }
        }
    }
}, pd.DataFrame([p for p in part_by_port]))

In [ ]:
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Distribution des tonnages cumulés des ports de la DFLR en 1789",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "port",
            "type": "ordinal", 
            "title": "port"
        },
        "y": {
            "type": "quantitative",
            "field": "portion_tonnage",
            "title": "flux sortants"
        },
        "color": {
            "type": "nominal",
            "field": "type",
            "title": "Type de quantification",
            "scale": {
                 "range": ["blue"]
            }
        }
    }
}, pd.DataFrame([p for p in part_by_port]))

In [ ]:
with open('csv_outputs/tableau_1', 'w', newline='') as csvfile:
        fieldnames = ['port', 'nb_pointcalls_out', 'cumulated_tonnage']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for port, values in ports.items(): # port est une key, values est un dictionnaire python
            writer.writerow(values)